In [1]:
import numpy as np
import pandas as pd
import gzip
import csv
import io
import argparse

import subprocess
import multiprocessing
import time

import sys
sys.path.append("/home/rashika/CAFA4/InformationAccretion/")
from ia import *
from make_benchmarks import *

In [2]:
# Preprocess
#preprocess_cmd = "python3 run_preprocess_parallel.py"
#result = subprocess.run(preprocess_cmd, shell=True)

In [3]:
# To do:
# Modify preprocess_cmd to take all the input/output/log paths as inputs
# Incorporate in the code of main() block

# To do 

Do the system commands more elegantly, parallelise the evaluation, incorporate the bootstrapping

In [4]:
"""
    Read a the processed annotation file, map the primary ID to a mapping file, and keep the rows that can be mapped.

    Parameters:
    - processed_file_path: Path to the file containing processed GOA annotations
    - mapping_file: Path to the mapping file 
    - out_path: Path to the output file.
    """
def goa_to_CAFA4ID(processed_file_path, mapping_file, out_path):
    ann = pd.read_csv(processed_file_path, sep = '\t', header = 0)
    mapping = pd.read_csv(mapping_file, sep = '\t', header = 0)

    # Inner join the processed annotations and the mapping based on DB Object ID
    mapped =  pd.merge(ann, mapping, on='DB Object ID', how='inner')
    
    
    # Keep the required columns
    mapped = mapped[["CAFA4_ID", "GO ID", "Aspect"]]
    
    # Write the mapped file to the out_path
    mapped.to_csv(out_path, sep = "\t", index=False, header=None)

In [6]:
def get_preprocess_cmd(gaf_path, out_path):
    cmd = [
    "python3",                 # Command to execute Python 3
    "preprocess_gaf.py",       # Script to run
    gaf_path,  # Path to input file
    "--highTP",
    "--out_path", out_path,        # Output path parameter
    #"--evidence_codes", "EXP", "IDA",   # Evidence codes parameter
    #"--extract_col_list", "DB Object ID", "Qualifier"  # Extract column list parameter
]
    return cmd

def run_process(command, log_file):
    with open(log_file, "w") as f:
        print(command)
        result = subprocess.run(" ".join(command), shell=True, stdout=f, stderr=subprocess.STDOUT)


if __name__ == "__main__":
    # Define commands and log file names
    work_dir = "/data/rashika/CAFA4/"
    
    #t0_gaf_file = work_dir + "uniprot/raw_goa/sample_t0.gz"
    t0_gaf_file = work_dir + "uniprot/raw_goa/t0/goa_uniprot_all.gaf.195.gz"
    t0_processed = work_dir + "extracted_goa/t0_preprocessed.csv"
    log_t0 =  work_dir + "log/log_preprocess_t0.txt"
    
    t1_gaf_file = work_dir + "uniprot/raw_goa/t1/goa_uniprot_all.gaf.gz"
    t1_processed = work_dir + "extracted_goa/t1_preprocessed.csv"
    log_t1 = work_dir + "log/log_preprocess_t1.txt"
    
    
    cmd_preprocess_t0 = get_preprocess_cmd(t0_gaf_file, t0_processed)
    cmd_preprocess_t1 = get_preprocess_cmd(t1_gaf_file, t1_processed)
    
    # Create processes for each command
    process1 = multiprocessing.Process(target=run_process, args=(cmd_preprocess_t0, log_t0))
    process2 = multiprocessing.Process(target=run_process, args=(cmd_preprocess_t1, log_t1))
    
    # Start the processes
    #process1.start()
    #process2.start()

    # Wait for both processes to finish
    #process1.join()
    #process2.join()

    print("Both processes have finished.")
    
    # Map the IDs of the processed 
    
    work_dir = "/data/rashika/CAFA4/"
    mapping_file = "/data/rashika/CAFA4/CAFA4-export/AC2CAFA4ID.map"
    t0_mapped_path = work_dir + "mapped/t0.csv"
    t1_mapped_path = work_dir + "/mapped/t1.csv"
    
    # Map to CAFA4 IDs 
    #goa_to_CAFA4ID(t0_processed , mapping_file, t0_mapped_path)
    #goa_to_CAFA4ID(t1_processed , mapping_file, t1_mapped_path)
    
    # Create the benchmarks
    roots = {'BPO': 'GO:0008150', 'CCO': 'GO:0005575', 'MFO': 'GO:0003674'}

    eval_path = work_dir + "eval/"

    t0_ont_file = '/data/rashika/CAFA4/uniprot/goa_2020_Jan_03/go-basic.obo' # data-version: releases/2020-01-01
    t0_ont_graph = clean_ontology_edges(obonet.read_obo(t0_ont_file)) 
    

    
    t1_ont_file = "/data/rashika/CAFA4/uniprot/goa_2024-02-09/go-basic.obo"
    t1_ont_graph = clean_ontology_edges(obonet.read_obo(t1_ont_file)) # data-version: releases/2024-01-17

    # Create BM lists
    BM_GO_path = eval_path + "BM_GO/"
    
    #  create_bm_lists(t0_mapped_path, t1_mapped_path, t0_ont_graph, t1_ont_graph, roots, BM_GO_path)


    # Calculate IA
    IA_file =  work_dir + "eval/IA/IA.txt"
    print(IA_file)
    t0_common_path = '/data/rashika/CAFA4/common/t0.tsv'
    #cmd = 'python3 /home/rashika/CAFA4/InformationAccretion/ia.py --annot ' + t0_common_path + ' --graph '+ t0_ont_file + ' --outfile ' + IA_file + ' --prop' 
    #os.system(cmd)
    
    pred_dir = "/data/yisupeng/sharing/cafa4/all_models/"
    
    result_path = "/data/rashika/CAFA4/eval/" + "eval_results/"
    run_eval(BM_GO_path, pred_dir, t0_ont_file, IA_file, result_path, log_path = '/home/rashika/CAFA4/eval/log/')
    
    




Both processes have finished.
/data/rashika/CAFA4/eval/IA/IA.txt
Evaluating: bpo_all_type1.txt
python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py /data/rashika/CAFA4/uniprot/goa_2020_Jan_03/go-basic.obo /data/yisupeng/sharing/cafa4/all_models/ /data/rashika/CAFA4/eval/BM_GO/bpo_all_type1.txt -out_dir /data/rashika/CAFA4/eval/eval_results/bpo_all_type1/ -ia /data/rashika/CAFA4/eval/IA/IA.txt -prop max -th_step 0.01  -no_orphans  &
Evaluating: cco_all_type1.txt
python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py /data/rashika/CAFA4/uniprot/goa_2020_Jan_03/go-basic.obo /data/yisupeng/sharing/cafa4/all_models/ /data/rashika/CAFA4/eval/BM_GO/cco_all_type1.txt -out_dir /data/rashika/CAFA4/eval/eval_results/cco_all_type1/ -ia /data/rashika/CAFA4/eval/IA/IA.txt -prop max -th_step 0.01  -no_orphans  &
Evaluating: mfo_all_type1.txt
python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py /data/rashika/CAFA4/uniprot/goa_2020_Jan_03/go-basic.obo /data/yi

2024-05-07 22:03:10,299 [INFO ] Ontology: biological_process, total 29392, roots 1, leaves 13564, alternative_ids 1552
2024-05-07 22:03:10,306 [INFO ] Ontology: biological_process, total 29392, roots 1, leaves 13564, alternative_ids 1552
2024-05-07 22:03:10,433 [INFO ] Ontology: biological_process, total 29392, roots 1, leaves 13564, alternative_ids 1552
2024-05-07 22:03:10,484 [INFO ] Ontology: biological_process, total 29392, roots 1, leaves 13564, alternative_ids 1552
2024-05-07 22:03:10,494 [INFO ] Ontology: biological_process, total 29392, roots 1, leaves 13564, alternative_ids 1552
2024-05-07 22:03:10,553 [INFO ] Ontology: biological_process, total 29392, roots 1, leaves 13564, alternative_ids 1552
2024-05-07 22:03:10,646 [INFO ] Ontology: biological_process, total 29392, roots 1, leaves 13564, alternative_ids 1552
2024-05-07 22:03:10,694 [INFO ] Ontology: biological_process, total 29392, roots 1, leaves 13564, alternative_ids 1552
2024-05-07 22:03:10,707 [INFO ] Ontology: biolog

2024-05-07 22:04:24,507 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M022, evaluated
2024-05-07 22:04:26,419 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M118, evaluated
2024-05-07 22:04:26,674 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M022, evaluated
2024-05-07 22:04:27,224 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M022, cellular_component, proteins 2111, annotations 139143, replaced alt. ids 44
2024-05-07 22:04:35,844 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M022, evaluated
2024-05-07 22:04:42,301 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M118, evaluated
2024-05-07 22:04:42,444 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M022, molecular_function, proteins 1971, annotations 215489, replaced alt. ids 156
2024-05-07 22:04:44,439 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M118, evaluated
2024-05-07 22:04:49,612 [INFO ] Prediction: /data/yisupeng/sharing/

2024-05-07 22:07:03,179 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M018, evaluated
2024-05-07 22:07:19,792 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M056, cellular_component, proteins 2111, annotations 817397, replaced alt. ids 0
2024-05-07 22:07:20,411 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M012, molecular_function, proteins 2300, annotations 55131, replaced alt. ids 0
2024-05-07 22:07:22,567 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M012, evaluated
2024-05-07 22:07:23,791 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M018, biological_process, proteins 1220, annotations 747039, replaced alt. ids 768
2024-05-07 22:07:26,793 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M046, evaluated
2024-05-07 22:07:28,560 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M056, evaluated
2024-05-07 22:07:31,525 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M046, biological_pro

2024-05-07 22:09:19,217 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M052, evaluated
2024-05-07 22:09:19,558 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M132, cellular_component, proteins 27, annotations 270, replaced alt. ids 0
2024-05-07 22:09:22,926 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M018, evaluated
2024-05-07 22:09:24,995 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M056, evaluated
2024-05-07 22:09:27,761 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M132, evaluated
2024-05-07 22:09:31,753 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M046, molecular_function, proteins 11427, annotations 753704, replaced alt. ids 36669
2024-05-07 22:09:33,185 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M095, cellular_component, proteins 1021, annotations 65401, replaced alt. ids 0
2024-05-07 22:09:36,367 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M095, cellular_compone

...

2024-05-07 22:22:56,021 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M080, cellular_component, proteins 1029, annotations 93524, replaced alt. ids 3551
2024-05-07 22:23:00,157 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M080, evaluated
2024-05-07 22:23:02,070 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M107, evaluated
2024-05-07 22:23:02,505 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M045, evaluated
2024-05-07 22:23:02,526 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M104, evaluated
2024-05-07 22:23:03,960 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M080, cellular_component, proteins 1082, annotations 96295, replaced alt. ids 3710
2024-05-07 22:23:06,322 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M049, cellular_component, proteins 1029, annotations 68527, replaced alt. ids 16
2024-05-07 22:23:08,361 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M080, evaluated
2

2024-05-07 22:25:51,455 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M027, evaluated
2024-05-07 22:25:56,185 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M049, evaluated
2024-05-07 22:25:59,031 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M121, molecular_function, proteins 351, annotations 18860, replaced alt. ids 2
2024-05-07 22:26:10,440 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M077, evaluated
2024-05-07 22:26:15,939 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M044, molecular_function, proteins 307, annotations 42019, replaced alt. ids 28
2024-05-07 22:26:16,193 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M031, molecular_function, proteins 9779, annotations 183775, replaced alt. ids 0
2024-05-07 22:26:20,347 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M121, evaluated
2024-05-07 22:26:22,111 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M141, molecular_function

2024-05-07 22:28:56,803 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M090, evaluated
2024-05-07 22:28:57,783 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M119, evaluated
2024-05-07 22:28:58,463 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M067, evaluated
2024-05-07 22:28:59,338 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M138, evaluated
2024-05-07 22:29:05,911 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M090, cellular_component, proteins 1082, annotations 22059, replaced alt. ids 0
2024-05-07 22:29:10,172 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M090, evaluated
2024-05-07 22:29:16,613 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M032, evaluated
2024-05-07 22:29:20,340 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M076, cellular_component, proteins 13208, annotations 1939654, replaced alt. ids 0
2024-05-07 22:29:21,128 [INFO ] Prediction: /data/yisupeng/sharing/ca

2024-05-07 22:31:39,627 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M091, cellular_component, proteins 1082, annotations 22059, replaced alt. ids 0
2024-05-07 22:31:42,743 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M057, cellular_component, proteins 1029, annotations 74606, replaced alt. ids 26
2024-05-07 22:31:43,345 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M119, cellular_component, proteins 1798, annotations 98342, replaced alt. ids 46
2024-05-07 22:31:44,113 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M091, evaluated
2024-05-07 22:31:44,710 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M122, molecular_function, proteins 339, annotations 19672, replaced alt. ids 2
2024-05-07 22:31:44,984 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M064, biological_process, proteins 889, annotations 79365, replaced alt. ids 1
2024-05-07 22:31:45,373 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_mod

2024-05-07 22:33:05,926 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M123, cellular_component, proteins 443, annotations 26012, replaced alt. ids 3
2024-05-07 22:33:07,797 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M106, evaluated
2024-05-07 22:33:09,423 [INFO ] Prediction: /data/yisupeng/sharing/cafa4/all_models/M107, molecular_function, proteins 1414, annotations 16996, replaced alt. ids 0


In [ ]:
#Bootstrapping


In [ ]:
t0_common_path = '/data/rashika/CAFA4/common/t0.tsv'
t0_ont_file = '/data/rashika/CAFA4/uniprot/goa_2020_Jan_03/go-basic.obo' # data-version: releases/2020-01-01
IA_file =  work_dir + "eval/IA/IA.txt"
cmd = 'python3 /home/rashika/CAFA4/InformationAccretion/ia.py --annot ' + t0_common_path + ' --graph '+ t0_ont_file + ' --outfile ' + IA_file + ' --prop &' 
os.system(cmd)

In [ ]:
roots = {'BPO': 'GO:0008150', 'CCO': 'GO:0005575', 'MFO': 'GO:0003674'}

work_dir = "/data/rashika/CAFA4/"
t0_mapped_path = work_dir + "mapped/t0.csv"
t1_mapped_path = work_dir + "/mapped/t1.csv"

t0_ont_file = '/data/rashika/CAFA4/uniprot/goa_2020_Jan_03/go-basic.obo' # data-version: releases/2020-01-01
t0_ont_graph = clean_ontology_edges(obonet.read_obo(t0_ont_file)) 
    
t1_mapped_ann =  "/data/rashika/CAFA4/mapped/t1_mapped.csv"
t1_ont_graph = clean_ontology_edges(obonet.read_obo( "/data/rashika/CAFA4/uniprot/goa_2024-02-09/go-basic.obo")) # data-version: releases/2024-01-17
    
#Prop t0 and t1 in their respective ontologies
t0_prop = process_raw_annot(t0_mapped_path, t0_ont_graph, roots, remove_roots = False)
t1_prop = process_raw_annot(t1_mapped_path, t1_ont_graph, roots, remove_roots = False)
    
# Keep common terms
t0_common, t1_common =  keep_common_go_terms(t0_prop, t1_prop, t0_ont_graph, t1_ont_graph)
t0_common.to_csv('/data/rashika/CAFA4/common/t0.tsv', sep = '\t', header = False, index = False)
t1_common.to_csv('/data/rashika/CAFA4/common/t1.tsv', sep = '\t', header = False, index = False)

In [ ]:
t0_common

In [ ]:
# Create the benchmarks
roots = {'BPO': 'GO:0008150', 'CCO': 'GO:0005575', 'MFO': 'GO:0003674'}

eval_path = "/data/rashika/CAFA4/eval/"

t0_ont_file = '/data/rashika/CAFA4/uniprot/goa_2020_Jan_03/go-basic.obo' # data-version: releases/2020-01-01
t0_ont_graph = clean_ontology_edges(obonet.read_obo(t0_ont_file)) 
t0_mapped_ann = "/data/rashika/CAFA4/mapped/t0_mapped.csv"
#t0_mapped_ann = "/data/rashika/CAFA4/t0_mapped_test.csv"

t1_mapped_ann =  "/data/rashika/CAFA4/mapped/t1_mapped.csv"
#t1_mapped_ann =  "/data/rashika/CAFA4/t1_mapped_test.csv"
t1_ont_graph = clean_ontology_edges(obonet.read_obo( "/data/rashika/CAFA4/uniprot/goa_2024-02-09/go-basic.obo")) # data-version: releases/2024-01-17

BM_GO_path = eval_path + "BM_GO/"

# Create BM lists
#create_bm_lists(t0_mapped_ann, t1_mapped_ann, t0_ont_graph, t1_ont_graph, roots, BM_GO_path)




In [ ]:
# Calculate Information Accretion
BM_GO_path = eval_path + "BM_GO/"
IA_path = "/data/rashika/CAFA4/eval/" + "IA/"
#pred_dir = "./one/" # One method
#pred_dir = "/data/yisupeng/sharing/cafa4/predictions/" # 5 methods
pred_dir = "/data/yisupeng/sharing/cafa4/all_models/" # all 144 methods
#pred_dir = "/data/rashika/CAFA4/pred_sample/" # 20 methods

#calc_IA(BM_GO_path, t0_ont_file, IA_path)

In [ ]:
# Run Evaluation
#run_eval(BM_GO_path, pred_dir, t0_ont_file, IA_path, result_path = '/home/rashika/CAFA4/eval/eval_results/', log_path = '/home/rashika/CAFA4/eval/log/')
#run_eval(BM_GO_path, pred_dir, t0_ont_file, IA_path, result_path = '/home/rashika/CAFA4/eval/eval_results_ALL/', log_path = '/home/rashika/CAFA4/eval/log/')

In [ ]:
result_path = "/data/rashika/CAFA4/eval/" + "eval_results/"
#run_eval(BM_GO_path, pred_dir, t0_ont_file, IA_path, result_path, log_path = '/home/rashika/CAFA4/eval/log/')

In [ ]:
# Obtain the plots
# Results of 5 methods
data_path = result_path
plots_path = "/data/rashika/CAFA4/eval/" + "plots/"
plots_path_f_w = plots_path+'f_w/'
plots_path_f = plots_path+'f/'
plots_path_f_micro_w = plots_path+'f_micro_w/'
plots_path_s_w = plots_path+'s_w/'
register = '/data/rashika/CAFA4/file_map.tsv'

metric, cols = ('f_w', ['rc_w', 'pr_w'])
#create_plots(results_path, metric, cols,out_path='/home/rashika/CAFA4/eval/plots/', n_curves = None, names_file = None):
create_plots(data_path, metric, cols, out_path = plots_path_f_w, n_curves = 10, names_file = register)

metric, cols = ('f', ['rc', 'pr'])
create_plots(data_path, metric, cols, out_path = plots_path_f, n_curves = 10, names_file = register)

metric, cols =  ('f_micro_w', ['rc_micro_w', 'pr_micro_w'])
create_plots(data_path, metric, cols, out_path = plots_path_f_micro_w, n_curves = 10, names_file = register)

metric, cols = ('s_w', ['ru_w', 'mi_w'])
create_plots(data_path, metric, cols, out_path = plots_path_s_w, n_curves = 10, names_file = register)



In [ ]:
# Bootstrapping

In [ ]:
def run_eval(BM_GO_path, pred_dir, ont_file, IA_path, result_path = '/home/rashika/CAFA4/eval/eval_results/', log_path = '/home/rashika/CAFA4/eval/log/'):
    dir_list = os.listdir(BM_GO_path) # out_path is the path to the folder containing the target GO sets

    if not os.path.exists(result_path):
        os.mkdir(result_path)
        
    if not os.path.exists(log_path):
        os.mkdir(log_path)
        
    for file in dir_list:

        print("Evaluating: " + file)
        out_dir = result_path + file.split(".")[0] + '/'
        IA_file = IA_path+'IA_'+ file.split(".")[0] + '.txt'
        if not os.path.exists(out_dir):
            os.mkdir(out_dir)
        
        # cafaeval go-basic.obo prediction_dir test_terms.tsv -ia IA.txt -prop fill -norm cafa -th_step 0.001 -max_terms 500
        #cmd = 'cafaeval /data/yisupeng/sharing/cafa4/gene_ontology_edit.obo.2020-01-01 /data/yisupeng/sharing/cafa4/all_models/ ' + '/data/yisupeng/sharing/cafa4/t1_truth.csv' + ' -out_dir '+ out_dir + ' -prop max -th_step 0.01  -no_orphans -log_level info > '+ log_path + file.split(".")[0] + '.log'+ ' &'
        #cmd = 'cafaeval '+ ont_file + pred_dir + BL_GO_path+file +' -out_dir '+ out_dir + ' -prop max -th_step 0.01  -no_orphans -log_level info > '+ log_path + file.split(".")[0] + '.log'+ ' &'
        #With IA
        cmd = "python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py "+ ont_file +" "+ pred_dir + " " + BM_GO_path+file + " -out_dir " + out_dir + ' -ia ' + IA_file + " -prop max -th_step 0.01  -no_orphans " + " &"
        #Without IA
        #cmd = "python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py "+ ont_file +" "+ pred_dir + " " + BM_GO_path+file + " -out_dir " + out_dir + " -prop max -th_step 0.01  -no_orphans " + " &"
        
        
        print(cmd)
        os.system(cmd)

In [ ]:
def create_plots(results_path, metric, cols,out_path='/home/rashika/CAFA4/eval/plots/', n_curves = None, names_file = None):
    dir_list = os.listdir(results_path)
    
    cumulate = True
    add_extreme_points = True
    coverage_threshold = 0.3
    axis_title_dict = {'pr': 'Precision', 'rc': 'Recall', 'f': 'F-score', 'pr_w': 'Weighted Precision', 'rc_w': 'Weighted Recall', 'f_w': 'Weighted F-score', 'mi': 'Misinformation (Unweighted)', 'ru': 'Remaining Uncertainty (Unweighted)', 'mi_w': 'Misinformation', 'ru_w': 'Remaining Uncertainty', 's': 'S-score', 'pr_micro': 'Precision (Micro)', 'rc_micro': 'Recall (Micro)', 'f_micro': 'F-score (Micro)', 'pr_micro_w': 'Weighted Precision (Micro)', 'rc_micro_w': 'Weighted Recall (Micro)', 'f_micro_w': 'Weighted F-score (Micro)'}
    ontology_dict = {'biological_process': 'BPO', 'molecular_function': 'MFO', 'cellular_component': 'CCO'}
    
    if not os.path.exists(out_path):
        os.mkdir(out_path)
    
    for file in dir_list:
        df_file = results_path + file +"/evaluation_all.tsv"
        df = pd.read_csv(df_file, sep="\t")
        out_folder = out_path + file
        if not os.path.exists(out_folder):
            os.mkdir(out_folder)
            
        
        df = pd.read_csv(df_file, sep="\t")
        
        # Set method information (optional)
        if names_file is None:
            df['group'] = df['filename']
            df['label'] = df['filename']
            df['is_baseline'] = False
        else:
            methods = pd.read_csv(names_file, sep = "\t", header=0)
            df = pd.merge(df, methods, on='filename', how='left')
            df['group'].fillna(df['filename'], inplace=True)
            df['label'].fillna(df['filename'], inplace=True)
            if 'is_baseline' not in df:
                df['is_baseline'] = False
            else:
                df['is_baseline'].fillna(False, inplace=True)
            # print(methods)
        #df = df.drop(columns='filename').set_index(['group', 'label', 'ns', 'tau'])
        df = df.set_index(['group', 'label', 'ns', 'filename','tau'])
        
        # Filter by coverage
        df = df[df['cov'] >= coverage_threshold]
        
        # Assign colors based on group
        cmap = plt.get_cmap('tab20')
        df['colors'] = df.index.get_level_values('group')
        df['colors'] = pd.factorize(df['colors'])[0]
        df['colors'] = df['colors'].apply(lambda x: cmap.colors[x % len(cmap.colors)])
        
        index_best = df.groupby(level=['group', 'ns'])[metric].idxmax() if metric in ['f', 'f_w', 'f_micro', 'f_micro_w'] else df.groupby(['group', 'ns'])[metric].idxmin()
        
        # Filter the dataframe for the best methods
        df_methods = df.reset_index('tau').loc[[ele[:-1] for ele in index_best], ['tau', 'cov', 'colors'] + cols + [metric]].sort_index()

        # Makes the curves monotonic. Cumulative max on the last column of the cols variable, e.g. "pr" --> precision
        if cumulate:
            if metric in ['f', 'f_w', 'f_micro', 'f_micro_w']:
                df_methods[cols[-1]] = df_methods.groupby(level=['label', 'ns'])[cols[-1]].cummax()
            else:
                df_methods[cols[-1]] = df_methods.groupby(level=['label', 'ns'])[cols[-1]].cummin()


        # Save to file
        df_methods.drop(columns=['colors']).to_csv('{}/fig_{}.tsv'.format(out_folder, metric), float_format="%.3f", sep="\t")
        
        # Add first last points to precision and recall curves to improve APS calculation
        def add_points(df_):
            df_ = pd.concat([df_.iloc[0:1], df_])
            df_.iloc[0, df_.columns.get_indexer(['tau', cols[0], cols[1]])] = [0, 1, 0]  # tau, rc, pr
            df_ = pd.concat([df_, df_.iloc[-1:]])
            df_.iloc[-1, df_.columns.get_indexer(['tau', cols[0], cols[1]])] = [1.1, 0, 1]
            return df_

        if metric.startswith('f') and add_extreme_points:
            df_methods = df_methods.reset_index().groupby(['group', 'label', 'ns'], as_index=False).apply(add_points).set_index(['group', 'label', 'ns'])
        
        # Filter the dataframe for the best method and threshold
        df_best = df.loc[index_best, ['cov', 'colors'] + cols + [metric]]
        
        # Calculate average precision score 
        if metric.startswith('f'):
            df_best['aps'] = df_methods.groupby(level=['group', 'label', 'ns'])[[cols[0], cols[1]]].apply(lambda x: (x[cols[0]].diff(-1).shift(1) * x[cols[1]]).sum())

        # Calculate the max coverage across all thresholds
        df_best['max_cov'] = df_methods.groupby(level=['group', 'label', 'ns'])['cov'].max()
        
        # Set a label column for the plot legend
        df_best['label'] = df_best.index.get_level_values('label')
        if 'aps' not in df_best.columns:
            df_best['label'] = df_best.agg(lambda x: f"{x['label']} ({metric.upper()}={x[metric]:.3f} C={x['max_cov']:.3f})", axis=1)
        else:
            df_best['label'] = df_best.agg(lambda x: f"{x['label']} ({metric.upper()}={x[metric]:.3f} APS={x['aps']:.3f} C={x['max_cov']:.3f})", axis=1)
        
        # Generate the figures
        plt.rcParams.update({'font.size': 22, 'legend.fontsize': 18})

        # F-score contour lines
        x = np.arange(0.01, 1, 0.01)
        y = np.arange(0.01, 1, 0.01)
        X, Y = np.meshgrid(x, y)
        Z = 2 * X * Y / (X + Y)

        
        for ns, df_g in df_best.groupby(level='ns'):
            fig, ax = plt.subplots(figsize=(15, 15))

             # Contour lines. At the moment they are provided only for the F-score
            if metric.startswith('f'):
                CS = ax.contour(X, Y, Z, np.arange(0.1, 1.0, 0.1), colors='gray')
                ax.clabel(CS, inline=True) #, fontsize=10)

            cnt = 0
            # Iterate methods
            for i, (index, row) in enumerate(df_g.sort_values(by=[metric, 'max_cov'], ascending=[False if metric.startswith('f') else True, False]).iterrows()):
                
                #data = df_methods.loc[index[:-1]]
                data = df_methods.loc[index[:-2]]
                print(row[cols[0]], row[cols[1]])

                # Precision-recall or mi-ru curves
                ax.plot(data[cols[0]], data[cols[1]], color=row['colors'], label=row['label'], lw=2, zorder=500-i)

                # F-max or S-min dots
                ax.plot(row[cols[0]], row[cols[1]], color=row['colors'], marker='o', markersize=12, mfc='none', zorder=1000-i)
                ax.plot(row[cols[0]], row[cols[1]], color=row['colors'], marker='o', markersize=6, zorder=1000-i)

                cnt+=1
                if n_curves and cnt >= n_curves:
                    break
                
            # Set axes limit
            if metric.startswith('f'):
                plt.xlim(0, 1)
                plt.ylim(0, 1)

            # plt.xlim(0, max(1, df_best.loc[:,:,ns,:][cols[0]].max()))
            # plt.ylim(0, max(1, df_best.loc[:,:,ns,:][cols[1]].max()))

            # Set titles
            ax.set_title(file)
            ax.set_xlabel(axis_title_dict[cols[0]], labelpad=20)
            ax.set_ylabel(axis_title_dict[cols[1]], labelpad=20)

            # Legend
            # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
            leg = ax.legend(markerscale=6, title=file)
            for legobj in leg.get_lines():
                legobj.set_linewidth(10.0)
                
            leg.set_bbox_to_anchor((1.05, 1))  

            # Save figure on disk
            plt.savefig("{}/fig_{}_{}.png".format(out_folder, metric, ns), bbox_inches='tight', dpi=300, transparent=True)
            # plt.clf()



In [ ]:
"""
    Read a CSV file in chunks, map the primary ID to a mapping file, and keep the rows that can be mapped.

    Parameters:
    - file_path: Path to the CSV file.
    - mapping_file: Path to the mapping file (CSV).
    - primary_id_column: Index of the column containing the primary ID in the mapping file.
    - out_path: Path to the output file.
    - chunk_size: Size of each chunk. Defaults to 100,000 lines.
    """
def map_goa_to_cafa_ids(file_path, mapping_file, primary_id_column, out_path, chunk_size=100000):
    # Read the mapping file into a DataFrame
    mapping_df = pd.read_csv(mapping_file, sep = ",", header = 0)
    mapping_df.columns = ["DB Object ID", "CAFA4_ID"]

    # Extract the primary IDs from the mapping file and convert to a set for efficient lookup
    id_set = set(mapping_df["DB Object ID"])

    # Initialize an empty list to store filtered chunk dataframes
    dfs = []

    # Read the CSV file in chunks
    #flag = 0
    for chunk in pd.read_csv(file_path, chunksize=chunk_size, sep = "\t"):
        # Filter the chunk based on whether the primary ID can be found in the mapping file
        filtered_chunk = chunk[chunk.iloc[:,primary_id_column].isin(id_set)]
        filtered_chunk = filtered_chunk.drop_duplicates().copy()
        dfs.append(filtered_chunk)

    # Concatenate all the filtered chunk dataframes into a single dataframe
    df = pd.concat(dfs, ignore_index=True)
    
    df_mapped = pd.merge(df, mapping_df, on='DB Object ID', how='inner')
    print("Rows in the mapped file : ", len(df_mapped))
    
    df_mapped = df_mapped.loc[:, ["CAFA4_ID", "GO ID", "Aspect"]].copy()

    # Write the final dataframe to the output file
    df_mapped.to_csv(out_path, index=False, sep = "\t")



In [ ]:
roots = {'BPO': 'GO:0008150', 'CCO': 'GO:0005575', 'MFO': 'GO:0003674'}

t0_ont_file = '/data/rashika/CAFA4/uniprot/goa_2020_Jan_03/go-basic.obo' # data-version: releases/2020-01-01
t0_ont_graph = clean_ontology_edges(obonet.read_obo(t0_ont_file)) 
shawn_t0_mapped_ann = "/data/rashika/CAFA4/CAFA4_gt/t0_mapped.csv"

t1_mapped_ann = "/data/rashika/CAFA4/CAFA4_gt/t1_mapped.csv"
t1_ont_graph = clean_ontology_edges(obonet.read_obo( "/data/rashika/CAFA4/uniprot/goa_2024-02-09/go-basic.obo")) # data-version: releases/2024-01-17

BM_path = "/home/rashika/CAFA4/eval/benchmarks/"
BM_GO_path = "/home/rashika/CAFA4/eval/benchmarks_GO/"

# Create BM lists
#t1_eval = create_bm_lists(shawn_t0_mapped_ann, t1_mapped_ann, t0_ont_graph, t1_ont_graph, roots, BM_path)




In [ ]:
import os

In [ ]:
def generate_image_code(directory):
        image_code = ''
        for root, dirs, files in os.walk(directory):
            for sub_dir in dirs:
                print(sub_dir)
                print(directory +sub_dir)
                for _, _, files_sub_dir in os.walk(directory +sub_dir):
                    for file in files_sub_dir:
                        print(str(file))
                        if file.endswith(".png"):
                            image_path = os.path.join(root, file)
                            folder = os.path.basename(root)
                            image_code += "\\begin{subfigure}[b]{0.3\\textwidth}\n"
                            image_code += "\\centering\n"
                            image_code += "\\includegraphics[width=\\textwidth]{" + image_path + "}\n"
                            image_code += "\\caption{Figure in \\texttt{" + folder + "}}\n"
                            image_code += "\\label{fig:" + folder + ":" + file + "}\n"
                            image_code += "\\end{subfigure}\n"
            return image_code



In [ ]:
# Call function to generate image code for current directory
latex_image_code = generate_image_code('/home/rashika/CAFA4/eval/plots_ALL/f/')

In [ ]:
latex_image_code